# Đánh giá Legal RAG System với bộ dữ liệu ViBidLQA


## Bước 1. Cài đặt và import

In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from tqdm.auto import tqdm
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

from legal_rag import LegalRAGSystem

# Tải biến môi trường (ví dụ: GOOGLE_API_KEY)
load_dotenv()

# Khởi tạo hệ thống RAG
rag_system = LegalRAGSystem()
if not rag_system.load_knowledge_base():
    raise RuntimeError("Knowledge base chưa sẵn sàng. Hãy chạy 'python rebuild_kb.py' trước rồi thử lại.")
print("✅ Hệ thống LegalRAG đã sẵn sàng.")

e:\Individual\ChatBot\law_chatbot\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
e:\Individual\ChatBot\law_chatbot\.venv\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.6) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
e:\Individual\ChatBot\law_chatbot\.venv\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.6) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please up

🔄 Using HuggingFace embeddings model: sentence-transformers/all-MiniLM-L6-v2
🔄 Using Google chat model: gemini-2.0-flash
🔄 Đang load vectorstore...
✅ Đã load knowledge base thành công!
✅ Hệ thống LegalRAG đã sẵn sàng.
🔄 Using Google chat model: gemini-2.0-flash
🔄 Đang load vectorstore...
✅ Đã load knowledge base thành công!
✅ Hệ thống LegalRAG đã sẵn sàng.


In [2]:
# Tải dữ liệu từ các file CSV
try:
    train = pd.read_csv("ViBidLQA/train.csv")
    dev = pd.read_csv("ViBidLQA/dev.csv")
    test = pd.read_csv("ViBidLQA/test.csv")
    print("Tải dữ liệu thành công.")
    test.head()
except FileNotFoundError:
    print("Lỗi: Không tìm thấy thư mục 'ViBidLQA' hoặc các file csv. Hãy chắc chắn bạn đã chạy cell để tải và lưu dữ liệu từ Hugging Face trước đó.")

Tải dữ liệu thành công.


In [3]:
test.head()

,context,question,answer
0,Luật Đấu thầu số 22/2023/QH15 Điều 3...,Phương thức hai giai đoạn một túi hồ...,đấu thầu rộng rãi
1,Nghị định hướng dẫn Luật Đấu thầu...,Nghị định hướng dẫn Luật Đấu thầu...,Thông tin về kết quả thực hiện hợp...
2,Luật Đấu thầu số 22/2023/QH15 Điều 2...,Luật Đấu thầu số 22/2023/QH15 Điều 2...,Lựa chọn nhà thầu trong trường hợp ...
3,Nghị định hướng dẫn Luật Đấu thầu...,Căn cứ pháp lý nào quy định về trình...,Luật Đấu thầu
4,Nghị định hướng dẫn Luật Đấu thầu...,Theo Nghị định hướng dẫn Luật Đấu t...,"là việc mua trực tiếp hàng hóa, dịch..."


In [4]:
test['question'][0]

'Phương thức hai giai đoạn một túi hồ sơ được áp dụng trong trường hợp nào?'

## Bước 2. Sinh câu trả lời từ hệ thống RAG

In [6]:
# Lấy một mẫu nhỏ của bộ test để chạy nhanh hơn (ví dụ: 20 mẫu)
# Bạn có thể tăng số lượng này để có kết quả đánh giá chính xác hơn
test_sample = test.head(15)

results = []
print("Generating answers and retrieving contexts for evaluation...")
for index, row in tqdm(test_sample.iterrows(), total=test_sample.shape[0]):
    question = row['question']

    # Lấy kết quả từ RAG system
    qa_result = rag_system.query(question)
    generated_answer = qa_result.get("answer", "")

    # Truy xuất lại các context đầy đủ từ vector store
    retrieved_docs = rag_system.vector_store.similarity_search(question, k=5)
    retrieved_contexts = [doc.page_content for doc in retrieved_docs]

    results.append({
        "question": question,
        "generated_answer": generated_answer,
        "retrieved_contexts": retrieved_contexts,
        "ground_truth": row['answer']  # Câu trả lời gốc từ dataset
    })

# Chuyển kết quả thành DataFrame để dễ xem
results_df = pd.DataFrame(results)
print("Done. Sample of generated results:")
results_df.head()

Generating answers and retrieving contexts for evaluation...


100%|██████████| 15/15 [00:22<00:00,  1.50s/it]

Done. Sample of generated results:


,question,generated_answer,retrieved_contexts,ground_truth
0,Phương thức hai giai đoạn một túi hồ...,Tôi không tìm thấy thông tin về phương thức ha...,[3. Văn bản để nghị phê duyệt phương án lựa ch...,đấu thầu rộng rãi
1,Nghị định hướng dẫn Luật Đấu thầu...,Tôi không tìm thấy thông tin về Nghị định hướn...,"[đ) Phạm vi cung cấp, yêu cầu về kỹ thuật, điề...",Thông tin về kết quả thực hiện hợp...
2,Luật Đấu thầu số 22/2023/QH15 Điều 2...,Tôi không tìm thấy thông tin về Điều 29 của Lu...,[Chương X\nDIEU KHOẢN THỊ HANH\n\nĐiều 95. Hiệ...,Lựa chọn nhà thầu trong trường hợp ...
3,Căn cứ pháp lý nào quy định về trình...,"Tôi tìm thấy các quy định liên quan đến trình,...",[d) Phân công việc chưa đủ điều kiện lập kế ho...,Luật Đấu thầu
4,Theo Nghị định hướng dẫn Luật Đấu t...,"Tôi không tìm thấy thông tin về định nghĩa ""mu...",[e) Những tài liệu phát sinh trong quá trình s...,"là việc mua trực tiếp hàng hóa, dịch..."


In [7]:
results_df.to_csv("evaluation_results.csv", index=False, encoding="utf-8-sig")

In [11]:
results_df = pd.read_csv("evaluation_results.csv")
results = results_df.to_dict(orient='records')

## Bước 3. Đánh giá chất lượng với RAGAs

In [12]:
import os
from datasets import Dataset
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall

# --- [PHẦN QUAN TRỌNG: SỬ DỤNG WRAPPER MỚI] ---
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

# 1. Kiểm tra dữ liệu đầu vào (giữ nguyên)
if 'results' not in globals() or len(results) == 0:
    raise ValueError("Danh sách 'results' trống. Vui lòng chạy Bước 2 để tạo kết quả.")

if "GOOGLE_API_KEY" not in os.environ or not os.environ["GOOGLE_API_KEY"]:
    raise EnvironmentError("Thiếu GOOGLE_API_KEY. Hãy cập nhật file .env và load lại.")

In [13]:
# 2. Chuẩn bị Dataset
ragas_dataset_dict = {
    'question': [str(r['question']) for r in results],
    'answer': [str(r['generated_answer']) for r in results],
    'contexts': [[str(ctx) for ctx in r['retrieved_contexts']] for r in results],
    'ground_truth': [str(r['ground_truth']) for r in results]
}
ragas_dataset = Dataset.from_dict(ragas_dataset_dict)

print("\nEvaluating with RAGAs (sử dụng Gemini)...")

# 3. Khởi tạo LangChain Model & Embeddings (giữ nguyên)
gemini_llm = ChatGoogleGenerativeAI(
    model=os.getenv("GOOGLE_CHAT_MODEL", "gemini-2.0-flash"),
    temperature=0.0,
    google_api_key=os.environ["GOOGLE_API_KEY"],
)

gemini_embeddings = GoogleGenerativeAIEmbeddings(
    model=os.getenv("GOOGLE_EMBEDDING_MODEL", "models/embedding-001"),
    google_api_key=os.environ["GOOGLE_API_KEY"],
)

# 4. [SỬA LỖI] Bọc (Wrap) chúng lại để Ragas hiểu
ragas_llm = LangchainLLMWrapper(gemini_llm)
ragas_embeddings = LangchainEmbeddingsWrapper(gemini_embeddings)

# 5. Chạy đánh giá
score = evaluate(
    ragas_dataset,
    llm=ragas_llm,              # Truyền wrapper vào đây
    embeddings=ragas_embeddings, # Truyền wrapper vào đây
    metrics=[faithfulness, answer_relevancy, context_precision, context_recall],
)

print("\nEvaluation results:")
print(score)
score_df = score.to_pandas()
print(score_df.head())


Evaluating with RAGAs (sử dụng Gemini)...


Evaluating:   0%|          | 0/60 [01:24<?, ?it/s]
Exception in thread Thread-11:
Traceback (most recent call last):
  File "C:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "e:\Individual\ChatBot\law_chatbot\.venv\lib\site-packages\ragas\executor.py", line 95, in run
    results = self.loop.run_until_complete(self._aresults())
  File "C:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\asyncio\base_events.py", line 646, in run_until_complete
    return future.result()
  File "e:\Individual\ChatBot\law_chatbot\.venv\lib\site-packages\ragas\executor.py", line 83, in _aresults
    raise e
  File "e:\Individual\ChatBot\law_chatbot\.venv\lib\site-packages\ragas\executor.py", line 78, in _aresults
    r = await future
  File "C:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\asyncio\tasks.py", line 571, in _wait_for_one
    return f.result()  # May raise f.exception().
  File "e:\Individual\ChatBot\la

KeyboardInterrupt: 